In [1]:
import pymysql as sql

In [64]:
connection = sql.connect(
    host="localhost",
    user="root",
    password="343450",
    database="Nutrition_Paradox"
)

cursor = connection.cursor()

In [6]:
def create_tables():
    cursor.execute(""" CREATE TABLE IF NOT EXISTS obesity(
                   region TEXT,
                   gender TEXT,
                   year INTEGER,
                   low REAL,
                   high REAL,
                   mean_estimate REAL,
                   country TEXT,
                   age_group TEXT,
                   ci_width REAL,
                   obesity_level TEXT);
                   """)
    
    cursor.execute(""" CREATE TABLE IF NOT EXISTS malnutrition(
                   region TEXT,
                   gender TEXT,
                   year INTEGER,
                   low REAL,
                   high REAL,
                   mean_estimate REAL,
                   country TEXT,
                   age_group TEXT,
                   ci_width REAL,
                   malnutrition_level TEXT);
                   """)
    connection.commit()

create_tables()

In [11]:
import pandas as pd

df_obesity = pd.read_csv("df_obesity_cleaned.csv")
df_malnutrition = pd.read_csv("df_malnutrition_cleaned.csv")

In [ ]:
for _, row in df_obesity.iterrows():
    cursor.execute(""" INSERT INTO obesity(region,gender,year,low,high,mean_estimate,country,age_group,ci_width,obesity_level) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)""",(
        row['Region'],row['Gender'],row['Year'],row['Low'],row['High'],row['Mean_Estimate'],row['Country'],row['age_group'],row['CI_Width'],row['obesity_level']))
    
for _, row in df_malnutrition.iterrows():
    cursor.execute(""" INSERT INTO malnutrition(region,gender,year,low,high,mean_estimate,country,age_group,ci_width,malnutrition_level) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)""",(
        row['Region'],row['Gender'],row['Year'],row['Low'],row['High'],row['Mean_Estimate'],row['Country'],row['age_group'],row['CI_Width'],row['malnutrition_level']))
    
connection.commit()

In [ ]:
query = """SELECT region, AVG(obesity_level) AS avg_obesity FROM obesity WHERE year = 2022 GROUP BY region ORDER BY avg_obesity DESC LIMIT 5;"""
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(f"Region: {row[0]}, Avg Obesity: {row[1]:.2f}")

Region: Africa, Avg Obesity: 0.00
Region: Western Pacific, Avg Obesity: 0.00
Region: Low Income, Avg Obesity: 0.00
Region: South-East Asia, Avg Obesity: 0.00
Region: Eastern Mediterranean, Avg Obesity: 0.00


In [22]:
query = """SELECT country, AVG(mean_estimate) AS avg_obesity FROM obesity GROUP BY country ORDER BY avg_obesity DESC LIMIT 5;"""
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(f"Country: {row[0]}, Avg Obesity: {row[1]:.2f}")

Country: Micronesia, Federated States of, Avg Obesity: 13.84
Country: Qatar, Avg Obesity: 13.81
Country: Grenada, Avg Obesity: 13.77
Country: Saint Lucia, Avg Obesity: 13.42
Country: New Zealand, Avg Obesity: 13.37


In [24]:
query = """SELECT year, AVG(mean_estimate) AS avg_obesity FROM obesity WHERE country = 'India' GROUP BY year ORDER BY year;"""
cursor.execute(query)
results = cursor.fetchall()

results

((2012, 2.473178141666667),
 (2013, 2.6346529999999997),
 (2014, 2.7987494500000003),
 (2015, 2.972813325),
 (2016, 3.1595759500000002),
 (2017, 3.362174975),
 (2018, 3.5792918250000003),
 (2019, 3.809012941666666),
 (2020, 4.0511491416666665),
 (2021, 4.307815533333333),
 (2022, 4.579179425))

In [26]:
query ="""SELECT gender, AVG(mean_estimate) AS avg_obesity FROM obesity GROUP BY gender;"""
cursor.execute(query)
results = cursor.fetchall()
results

(('Male', 6.349850918075738),
 ('Both', 6.6214524228951),
 ('Female', 5.942496979788128))

In [27]:
query ="""SELECT age_group, obesity_level, COUNT(DISTINCT country) AS country_count FROM obesity GROUP BY age_group, obesity_level ORDER BY age_group, obesity_level;"""
cursor.execute(query)
results = cursor.fetchall()
results


(('adults', 'Low', 104), ('children', 'Low', 186))

In [ ]:
#Top 5 least reliable countries (highest average CI_Width)
query = """SELECT country, AVG(ci_width) AS avg_ci_width FROM obesity GROUP BY country ORDER BY avg_ci_width DESC LIMIT 5;"""
cursor.execute(query)
results = cursor.fetchall()
results

(('Andorra', 12.795517625000002),
 ('Iceland', 11.953148820833334),
 ('Saint Lucia', 11.635337199999997),
 ('Luxembourg', 11.123110580851062),
 ('Micronesia, Federated States of', 10.8340506))

In [ ]:
#Top 5 most consistent countries (lowest average CI_Width)
query = """SELECT country, AVG(ci_width) AS avg_ci_width FROM obesity GROUP BY country ORDER BY avg_ci_width ASC LIMIT 5;"""
cursor.execute(query)
results = cursor.fetchall()
results

(('GLOBAL', 0.7205499316239319),
 ('WB_LMI', 0.8527930569230767),
 ('SEAR', 1.0177745681818184),
 ('India', 1.1401325691666657),
 ('Japan', 1.1559030522727274))

In [30]:
query = """SELECT age_group, AVG(obesity_level) AS avg_obesity FROM obesity GROUP BY age_group ORDER BY avg_obesity DESC;"""
cursor.execute(query)
results = cursor.fetchall()
results

(('adults', 0.0), ('children', 0.0))

In [32]:
query = """SELECT country,AVG(mean_estimate) AS avg_obesity,AVG(ci_width) AS avg_ci_width FROM obesity GROUP BY country ORDER BY avg_obesity ASC, avg_ci_width ASC LIMIT 10;"""
cursor.execute(query)
results = cursor.fetchall()
results

(('Timor-Leste', 0.9940674262878791, 1.273569166742424),
 ('Ethiopia', 1.4919943762878793, 1.5894139543939392),
 ('Burkina Faso', 1.721495976818182, 1.531513891136364),
 ('Rwanda', 1.7671937959090906, 2.36057863455303),
 ('Madagascar', 1.8622618037795275, 3.9090279261968504),
 ('Bangladesh', 1.9766725870454547, 1.6635064317424246),
 ('Tajikistan', 2.1091290047619045, 1.859243462857143),
 ('Burundi', 2.195447030708661, 4.751506403858268),
 ('Malawi', 2.391323871439393, 2.6327135187878787),
 ('Nepal', 2.4942547262878794, 3.098050873560607))

In [33]:
query = """SELECT f.country,f.year,f.mean_estimate AS female_obesity,m.mean_estimate AS male_obesity,(f.mean_estimate - m.mean_estimate) AS diff FROM obesity f
JOIN obesity m ON f.country = m.country AND f.year = m.year AND f.age_group = m.age_group WHERE f.gender = 'Female' AND m.gender = 'Male' AND (f.mean_estimate - m.mean_estimate) > 5 -- threshold for "large margin" ORDER BY diff DESC;"""
cursor.execute(query)
results = cursor.fetchall()
results

(('AFR', 2018, 13.587286, 5.095024, 8.492262),
 ('Gambia', 2014, 12.350465, 5.4043762, 6.9460888),
 ('Uganda', 2020, 8.8652313, 2.8132528, 6.0519785),
 ("Côte d'Ivoire", 2013, 10.251197, 4.957065, 5.294131999999999),
 ('Congo', 2021, 11.971641, 3.5714678, 8.4001732),
 ('Malawi', 2022, 10.003252, 2.3371365, 7.6661155),
 ('WB_LI', 2022, 11.289261, 5.8158137, 5.4734473),
 ('Zimbabwe', 2013, 13.793187, 2.554643, 11.238544),
 ('Central African Republic', 2020, 9.5557496, 4.4378806, 5.117869000000001),
 ('Zambia', 2021, 13.72315, 3.8448852, 9.8782648),
 ('Kenya', 2014, 11.694031, 3.3115461, 8.382484900000001),
 ('Senegal', 2021, 13.145837, 3.2044102, 9.9414268),
 ('South Sudan', 2019, 10.009216, 3.8804203, 6.1287957),
 ("Côte d'Ivoire", 2018, 12.308858, 5.8667652, 6.442092800000001),
 ('Congo', 2012, 10.361423, 2.6907656, 7.6706574),
 ('Indonesia', 2016, 11.652267, 4.293935, 7.358332),
 ('Uganda', 2018, 7.8797847, 2.2105558, 5.6692289),
 ('Togo', 2019, 13.569026, 4.3062184, 9.262807599999999

In [34]:
query = """SELECT year,AVG(mean_estimate) AS global_avg_obesity FROM obesity GROUP BY year ORDER BY year;"""
cursor.execute(query)
results = cursor.fetchall()
results

((2012, 5.9139809690690885),
 (2013, 5.98584073091507),
 (2014, 6.084495507702622),
 (2015, 6.190558185388916),
 (2016, 6.256076421486221),
 (2017, 6.326712241117689),
 (2018, 6.421416130040737),
 (2019, 6.515614170282488),
 (2020, 6.5828193515840185),
 (2021, 6.654720267270607),
 (2022, 6.703297767251652))

In [35]:
query = """SELECT age_group,AVG(mean_estimate) AS avg_malnutrition FROM malnutrition GROUP BY age_group ORDER BY avg_malnutrition DESC;"""
cursor.execute(query)
results = cursor.fetchall()
results

(('adults', 4.490448731084016), ('children', 4.197328697933834))

In [36]:
query ="""SELECT country,AVG(mean_estimate) AS avg_malnutrition FROM malnutrition GROUP BY country ORDER BY avg_malnutrition DESC LIMIT 5;"""
cursor.execute(query)
results = cursor.fetchall()
results

(('India', 14.138433499999998),
 ('SEAR', 13.704003999999998),
 ('WB_LMI', 13.625373633802813),
 ('Bangladesh', 13.609326333333335),
 ('Sri Lanka', 13.439482933333334))

In [37]:
query ="""SELECT year, AVG(mean_estimate) AS avg_malnutrition FROM malnutrition WHERE region = 'Africa' GROUP BY year ORDER BY year;"""
cursor.execute(query)
results=cursor.fetchall()
results

((2012, 7.367109442733196),
 (2013, 7.328449953333334),
 (2014, 7.23329489935484),
 (2015, 7.155785632974141),
 (2016, 7.091001585900214),
 (2017, 6.989837636784136),
 (2018, 6.93851088533333),
 (2019, 6.885143227130031),
 (2020, 6.790631605479452),
 (2021, 6.6872900649532685),
 (2022, 6.611304702631581))

In [39]:
query ="""SELECT gender,AVG(mean_estimate) AS avg_malnutrition FROM malnutrition GROUP BY gender;"""
cursor.execute(query)
results = cursor.fetchall()
results

(('Both', 4.563439056976306),
 ('Female', 4.0131982610114525),
 ('Male', 4.226449526242929))

In [40]:
query ="""SELECT malnutrition_level,age_group,AVG(ci_width) AS avg_ci_width FROM malnutrition GROUP BY malnutrition_level, age_group ORDER BY malnutrition_level, age_group;"""
cursor.execute(query)
results = cursor.fetchall()
results

(('Low', 'adults', 2.6155109489279695),
 ('Low', 'children', 4.047884662211968),
 ('Moderate', 'adults', 5.273872564653903),
 ('Moderate', 'children', 6.9294115757869195))

In [41]:
query = """SELECT country,year,AVG(mean_estimate) AS avg_malnutrition FROM malnutrition WHERE country IN ('India', 'Nigeria', 'Brazil') GROUP BY country, year ORDER BY country, year;"""
cursor.execute(query)
results=cursor.fetchall()
results

(('Brazil', 2012, 2.8386533000000003),
 ('Brazil', 2013, 2.7813881583333333),
 ('Brazil', 2014, 2.735034283333333),
 ('Brazil', 2015, 2.6989469583333334),
 ('Brazil', 2016, 2.6722411833333335),
 ('Brazil', 2017, 2.6535659499999995),
 ('Brazil', 2018, 2.641574491666667),
 ('Brazil', 2019, 2.6345942499999997),
 ('Brazil', 2020, 2.6294117166666666),
 ('Brazil', 2021, 2.626636775),
 ('Brazil', 2022, 2.626853608333333),
 ('India', 2020, 14.687687),
 ('India', 2021, 14.262235),
 ('India', 2022, 13.872814666666665),
 ('Nigeria', 2012, 9.971890140000001),
 ('Nigeria', 2013, 10.0273645),
 ('Nigeria', 2014, 10.10995364),
 ('Nigeria', 2015, 10.213868810000001),
 ('Nigeria', 2016, 10.33422092),
 ('Nigeria', 2017, 10.46389867),
 ('Nigeria', 2018, 10.59642715),
 ('Nigeria', 2019, 10.73518189),
 ('Nigeria', 2020, 10.387280311111112),
 ('Nigeria', 2021, 10.528758266666667),
 ('Nigeria', 2022, 10.4056174375))

In [42]:
query = """SELECT region,AVG(mean_estimate) AS avg_malnutrition FROM malnutrition GROUP BY region ORDER BY avg_malnutrition ASC LIMIT 5;"""
cursor.execute(query)
results=cursor.fetchall()
results

(('Americas Region', 2.024236674242424),
 ('High Income', 2.322331581060607),
 ('Europe', 2.4290010327830154),
 ('Western Pacific', 3.0434788099225867),
 ('Americas', 3.1284801282498456))

In [43]:
query = """SELECT country,MIN(mean_estimate) AS min_malnutrition,MAX(mean_estimate) AS max_malnutrition,
    (MAX(mean_estimate) - MIN(mean_estimate)) AS difference FROM malnutrition GROUP BY country HAVING MAX(mean_estimate) - MIN(mean_estimate) > 0 ORDER BY difference DESC;"""
cursor.execute(query)
results=cursor.fetchall()
results

(('Japan', 1.3626638, 13.270979, 11.9083152),
 ('Saudi Arabia', 3.1890925, 15.093192, 11.904099500000001),
 ('Haiti', 3.6696102, 15.199393, 11.5297828),
 ('Burkina Faso', 3.5992008, 15.108404, 11.5092032),
 ('Congo, The Democratic Republic of the', 3.988893, 15.275164, 11.286271),
 ('Kenya', 4.8048262, 15.232283, 10.427456800000002),
 ('Zimbabwe', 2.2208489, 12.500335, 10.2794861),
 ('Congo', 4.4272255, 14.605713, 10.1784875),
 ('Maldives', 5.228566, 15.273099, 10.044533000000001),
 ('Lesotho', 2.3480572, 12.089641, 9.7415838),
 ('Mozambique', 2.4245897, 12.147893, 9.7233033),
 ('Liberia', 1.8604903, 11.424839, 9.5643487),
 ('Tanzania, United Republic of', 3.5633728, 13.00149, 9.4381172),
 ('Zambia', 3.0486852, 12.365415, 9.316729800000001),
 ('EMR', 6.104935, 15.276098, 9.171163),
 ('WB_LI', 6.1365955, 15.256278, 9.1196825),
 ('AFR', 5.953339, 14.923282, 8.969943),
 ('Cambodia', 6.2305983, 15.010221, 8.779622700000001),
 ('Malawi', 1.6792592, 10.267731, 8.588471799999999),
 ('Iran, Is

In [44]:
query = """SELECT year,MIN(mean_estimate) AS min_malnutrition,MAX(mean_estimate) AS max_malnutrition FROM malnutrition GROUP BY year ORDER BY year;"""
cursor.execute(query)
results=cursor.fetchall()
results

((2012, 0.17530927, 15.273099),
 (2013, 0.17642054, 15.276098),
 (2014, 0.17777219, 15.216739),
 (2015, 0.17844464, 15.25128),
 (2016, 0.17880352, 15.237883),
 (2017, 0.17928582, 15.276808),
 (2018, 0.17979586, 15.204459),
 (2019, 0.18042778, 15.275164),
 (2020, 0.18198916, 15.125768),
 (2021, 0.18471746, 15.261832),
 (2022, 0.18817388, 15.225276))

In [ ]:
query = """SELECT country,region,year,ci_width,mean_estimate FROM malnutrition WHERE ci_width > 5 ORDER BY ci_width DESC;"""
cursor.execute(query)
results=cursor.fetchall()
results

(('Cambodia', 'Western Pacific', 2022, 13.1551394, 8.0324344),
 ('Niger', 'Africa', 2018, 13.155034699999998, 13.211338),
 ('Cabo Verde', 'Africa', 2020, 13.1492995, 8.4342711),
 ('Central African Republic', 'Africa', 2016, 13.1491045, 6.409584),
 ('Angola', 'Africa', 2012, 13.1478842, 10.307255),
 ('Suriname', 'Americas', 2016, 13.14678159, 5.0887056),
 ('Djibouti', 'Eastern Mediterranean', 2021, 13.14461684, 4.8124094),
 ('Saudi Arabia', 'Eastern Mediterranean', 2022, 13.1400478, 9.1339491),
 ('Myanmar', 'South-East Asia', 2016, 13.1385765, 8.9733258),
 ("Lao People's Democratic Republic",
  'Western Pacific',
  2017,
  13.1333196,
  8.6650744),
 ("Korea, Democratic People's Republic of",
  'South-East Asia',
  2021,
  13.1318943,
  9.2332728),
 ('Angola', 'Africa', 2019, 13.128710900000002, 9.3543954),
 ('Guinea', 'Africa', 2020, 13.1274065, 11.023823),
 ('Angola', 'Africa', 2015, 13.127029699999998, 11.822139),
 ('Botswana', 'Africa', 2016, 13.124861300000005, 12.932912),
 ('Benin'

In [47]:
query = """SELECT o.country,o.year,AVG(o.mean_estimate) AS avg_obesity,AVG(m.mean_estimate) AS avg_malnutrition
FROM obesity o JOIN malnutrition m ON o.country = m.country AND o.year = m.year
WHERE o.country IN ('India', 'Nigeria', 'Brazil', 'China', 'USA')GROUP BY o.country, o.year ORDER BY o.country, o.year;"""
cursor.execute(query)
results=cursor.fetchall()
results

(('Brazil', 2012, 9.885203371428563, 2.838653299999996),
 ('Brazil', 2013, 9.642166433333337, 2.7813881583333337),
 ('Brazil', 2014, 10.113106916666668, 2.7350342833333325),
 ('Brazil', 2015, 10.577600699999994, 2.6989469583333334),
 ('Brazil', 2016, 11.024693699999995, 2.672241183333335),
 ('Brazil', 2017, 10.901875299999999, 2.653565949999998),
 ('Brazil', 2018, 11.330340640000001, 2.641574491666667),
 ('Brazil', 2019, 11.757315119999998, 2.63459425),
 ('Brazil', 2020, 11.612804499999996, 2.6294117166666666),
 ('Brazil', 2021, 12.041788500000008, 2.6266367749999993),
 ('Brazil', 2022, 12.482320000000001, 2.6268536083333323),
 ('China', 2012, 6.201950774999999, 5.080563449999998),
 ('China', 2013, 6.666021624999992, 4.903683666666665),
 ('China', 2014, 7.131323083333334, 4.737267874999997),
 ('China', 2015, 6.973242972727278, 4.578603525000007),
 ('China', 2016, 7.421102954545461, 4.4279074583333315),
 ('China', 2017, 7.880929427272732, 4.286700691666657),
 ('China', 2018, 8.363828900

In [48]:
query = """SELECT o.country,o.year,o.gender,AVG(o.mean_estimate) AS avg_obesity,AVG(m.mean_estimate) AS avg_malnutrition,
    (AVG(o.mean_estimate) - AVG(m.mean_estimate)) AS disparity FROM obesity o JOIN malnutrition m 
    ON o.country = m.country AND o.year = m.year AND o.gender = m.gender GROUP BY o.country, o.year, o.gender ORDER BY disparity DESC;"""
cursor.execute(query)
results=cursor.fetchall()
results

(('New Zealand', 2017, 'Female', 14.061089, 0.85238404, 13.20870496),
 ('Kiribati', 2015, 'Both', 13.914894, 0.8828058950000001, 13.032088105),
 ('New Zealand', 2016, 'Female', 13.781962, 0.8737899625000001, 12.9081720375),
 ('Greece', 2012, 'Male', 14.04808, 1.2128668775, 12.8352131225),
 ('Australia', 2022, 'Female', 13.890409, 1.1530777799999998, 12.73733122),
 ('Greece', 2013, 'Male', 13.935848, 1.2003274875, 12.735520512499999),
 ('Micronesia, Federated States of',
  2012,
  'Both',
  13.840398,
  1.1174772825000001,
  12.722920717500001),
 ('Kiribati', 2014, 'Both', 13.572802, 0.8750129700000001, 12.69778903),
 ('New Zealand', 2015, 'Female', 13.502792, 0.89532531, 12.607466689999999),
 ('Greece', 2014, 'Male', 13.759521, 1.1987303175, 12.560790682499999),
 ('Australia',
  2017,
  'Male',
  14.085558,
  1.6110854749999999,
  12.474472525000001),
 ('Australia',
  2021,
  'Female',
  13.582310999999999,
  1.1623311399999998,
  12.41997986),
 ('Mexico', 2012, 'Both', 14.074044, 1.70

In [49]:
query = """SELECT year,AVG(CASE WHEN region = 'Africa' THEN mean_estimate END) AS africa_avg,
    AVG(CASE WHEN region = 'Americas Region' THEN mean_estimate END) AS americas_avg FROM obesity GROUP BY year ORDER BY year;"""
cursor.execute(query)
results=cursor.fetchall()
results

((2012, 3.548009504102561, 12.737475333333334),
 (2013, 3.6278132568333334, 13.022602),
 (2014, 3.7546874849532736, 13.1206482),
 (2015, 3.8795753635160684, 13.20047025),
 (2016, 4.018975119789275, 13.288460666666666),
 (2017, 4.116472385786406, 13.344555499999998),
 (2018, 4.23229898259406, 13.690321),
 (2019, 4.380346969272733, 13.772932),
 (2020, 4.4794405670378135, 14.111488),
 (2021, 4.584678358517702, None),
 (2022, 4.692976799364704, None))

In [61]:
query = """WITH obesity_change AS (SELECT country,MAX(mean_estimate) - MIN(mean_estimate) AS obesity_change FROM obesity GROUP BY country),
malnutrition_change AS (SELECT country,MAX(mean_estimate) - MIN(mean_estimate) AS malnutrition_change FROM malnutrition GROUP BY country)SELECT o.country,o.obesity_change,
m.malnutrition_change FROM obesity_change o JOIN malnutrition_change m ON o.country = m.country
WHERE o.obesity_change > 0 AND m.malnutrition_change < 0 ORDER BY o.obesity_change DESC, m.malnutrition_change ASC;"""
cursor.execute(query)
results=cursor.fetchall()
results

()

In [65]:
query = """SELECT o.year,o.age_group,AVG(o.mean_estimate) AS avg_obesity,AVG(m.mean_estimate) AS avg_malnutrition FROM obesity o
JOIN malnutrition m ON o.year = m.year AND o.age_group = m.age_group GROUP BY o.year, o.age_group ORDER BY o.year, o.age_group;"""
cursor.execute(query)
results=cursor.fetchall()
for row in results:
    print(row)

(2012, 'adults', 6.868346287621303, 4.627020012072906)
(2012, 'children', 5.754920082638, 4.205334115477139)
(2013, 'adults', 6.854253492178114, 4.579108131359781)
(2013, 'children', 5.844527388665302, 4.20832155318249)
(2014, 'adults', 6.877835750133231, 4.536595703502072)
(2014, 'children', 5.957785334709899, 4.22656267843203)
(2015, 'adults', 6.935599097945504, 4.548633139480917)
(2015, 'children', 6.073004900178722, 4.203618282027903)
(2016, 'adults', 7.10219433823265, 4.497445320803716)
(2016, 'children', 6.120923856775271, 4.212918439645349)
(2017, 'adults', 7.241488742665838, 4.463082729483938)
(2017, 'children', 6.180180536070546, 4.203483986642141)
(2018, 'adults', 7.429155970776558, 4.432322774291645)
(2018, 'children', 6.257568926254469, 4.211435112667811)
(2019, 'adults', 7.557179152049973, 4.496866907652197)
(2019, 'children', 6.344304140392433, 4.20773910184369)
(2020, 'adults', 7.690705353710997, 4.447575955052343)
(2020, 'children', 6.397054542235776, 4.172815529855725)